# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
# Load the csv file into a DF. 
VacationDF = pd.read_csv("weatherdata.csv", encoding="utf-8")
VacationDF.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pervomayskaya,43.4033,45.5261,69.60,78,0,2.24,RU,1659230262
1,Busselton,-33.6500,115.3333,56.14,70,79,10.58,AU,1659230263
2,Lugovoy,42.9472,72.7644,67.53,50,8,5.10,KZ,1659230263
3,Provideniya,64.3833,-173.3000,53.53,65,98,6.31,RU,1659230263
4,Saint Paul Harbor,57.7900,-152.4072,62.11,72,100,9.22,US,1659230264


In [48]:
# Figure out the data types for manipulation later.
VacationDF.dtypes

City                object
Latitude           float64
Longitude          float64
Max Temperature    float64
Humidity             int64
Cloudiness           int64
Wind Speed         float64
Country             object
Date                 int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [49]:
# Loads the gmaps configure protocol using my api key
gmaps.configure(api_key=g_key)

In [55]:
# Creates a new DF with just lat and long values and creates a list of humidity values.
Locations = VacationDF[["Latitude", "Longitude"]]
Humidity = VacationDF["Humidity"]
# Locations.head()
Humidity.head()

# Creates the heatmap figure
HeatMap = gmaps.figure()
# HeatMap - was to test that it loaded

In [54]:
# Creates a max value to set as the max intensity of the heatmap.
HumidityMax = Humidity.max()

# Creates a heat layer that maps humidity values over lats and logs and sets a max values by the max value
# in the data set.
HeatLayer = gmaps.heatmap_layer(LocationDF, weights=Humidity, dissipating=False, max_intensity=HumidityMax,point_radius=2)

HeatMap.add_layer(HeatLayer)

HeatMap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [77]:
# Create a DF that meets the following criteria:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

IdealWeatherDF = VacationDF.loc[(VacationDF["Max Temperature"] < 80) & (VacationDF["Max Temperature"] >=70)
                               & (VacationDF["Wind Speed"] < 10) & (VacationDF["Cloudiness"] == 0)]
# Next two lines reset the index and then delete the previous one. This sets up the for loop parameters for later.
IdealWeatherDF.reset_index(inplace=True)
del IdealWeatherDF['index']
IdealWeatherDF.head(22)
# print(len(IdealWeatherDF))

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Birobidzhan,48.8000,132.9500,78.78,75,0,4.25,RU,1659230265
1,Albany,42.6001,-73.9662,73.63,63,0,3.00,US,1659230135
2,Lata,40.1629,-8.3327,70.11,56,0,3.49,PT,1659230338
3,Praia da Vitória,38.7333,-27.0667,71.85,88,0,4.47,PT,1659230413
4,Fomboni,-12.2800,43.7425,75.63,72,0,3.83,KM,1659230422
5,Lagoa,39.0500,-27.9833,72.59,71,0,5.97,PT,1659230558
6,Michigan City,41.7075,-86.8950,76.87,77,0,0.00,US,1659230541
7,Hazorasp,41.3194,61.0742,71.44,37,0,8.68,UZ,1659230560
8,Carbonia,39.1672,8.5222,75.85,74,0,6.98,IT,1659230633
9,Ajdabiya,30.7554,20.2263,78.75,70,0,9.57,LY,1659230633


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
# Creates the new hotel_df, adds a blank column for Hotel Name.
hotel_df = IdealWeatherDF
hotel_df["Hotel Name"] = ""
hotel_df.head(22)

C:\Users\Stebo\AppData\Local\Temp\ipykernel_114552\3159555934.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Birobidzhan,48.8000,132.9500,78.78,75,0,4.25,RU,1659230265,
1,Albany,42.6001,-73.9662,73.63,63,0,3.00,US,1659230135,
2,Lata,40.1629,-8.3327,70.11,56,0,3.49,PT,1659230338,
3,Praia da Vitória,38.7333,-27.0667,71.85,88,0,4.47,PT,1659230413,
4,Fomboni,-12.2800,43.7425,75.63,72,0,3.83,KM,1659230422,
5,Lagoa,39.0500,-27.9833,72.59,71,0,5.97,PT,1659230558,
6,Michigan City,41.7075,-86.8950,76.87,77,0,0.00,US,1659230541,
7,Hazorasp,41.3194,61.0742,71.44,37,0,8.68,UZ,1659230560,
8,Carbonia,39.1672,8.5222,75.85,74,0,6.98,IT,1659230633,
9,Ajdabiya,30.7554,20.2263,78.75,70,0,9.57,LY,1659230633,


In [81]:
# Testing with Italian city above. Lodging seems best for hotels from the types field. 

ParamsTest = {
    "location": f"39.1672,8.5222",
    "radius": 5000,
    "types" : "lodging",
    "key": g_key
}

BaseURLTest = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
RequestTest = requests.get(BaseURLTest, params=ParamsTest)
RequestJSONTest = RequestTest.json()
RequestJSONTest

{'html_attributions': [],
 'next_page_token': 'AeJbb3c2RTq_--IlnOWcQIWvlUulCYOqqvLkhYUoHQBEDf97V1m1g4HXxsyg3dwyKI6Gs-ZXkNiVqOVvObIQKcs6ZNI1k59y_rL-s2l3m_860FcEpDH8dVYzqbi9iNLIBmFUca4S88vf4CkDOt5RsfZ6YVu4Gml2nhPC3N5vcmMsf9fjlhBVaBLudybQF3ST3N3IJNSxK0_2Yuvcrf6_nCrGF50aNuITInw6niwJDjfYEKSWco0ctgWmUwmwRmkAxGJvuER5jl8dOWdPliHeYXbFPITK_jaoJmde6wJa1inUhj-izvSwL2O6PnmF86rr4JjygTsL9X2wTvoFIvBQEtZvcYNo2YaCNxwqeXcUWAn07mUnM9j_nVdRYoAfh5Dy6eRNNTjiRSkRWxlUW2cDkGNwwrZkhW4Yvc074XnVQDGa_FWIWC-_NIncaopM',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 39.1674734, 'lng': 8.5163621},
    'viewport': {'northeast': {'lat': 39.1688955802915,
      'lng': 8.517743730291501},
     'southwest': {'lat': 39.1661976197085, 'lng': 8.515045769708497}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [92]:
# Creates an empty list to fill the hotels into.
Hotels = []

# For loop that iterates through the length of the hotel df to grab the closest hotel to ideal weather cities. 
for x in range(len(hotel_df)):
    Lats = hotel_df.loc[x]['Latitude']
    Longs = hotel_df.loc[x]['Longitude']
    # Parameters set to look up to 5000 meters away from latitude and longitude values. Within types, it looks for hotels.
    Params = {
        "location": f"{Lats},{Longs}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    
    BaseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    Request = requests.get(BaseURL, params=Params)
    RequestJSON = Request.json()
    # Uses a try / except loop to append hotels to the ideal weather cities DF. Grabs the closest one.
    try:
        Hotels.append(RequestJSON['results'][0]['name'])
    except:
        # This stores an empty value to keep line integrity. 
        Hotels.append("No Hotel")
hotel_df["Hotel Name"] = Hotels

hotel_df.head(22)

,level_0,index,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,0,Birobidzhan,48.8000,132.9500,78.78,75,0,4.25,RU,1659230265,Hotel Central (Vostok)
1,1,2,Lata,40.1629,-8.3327,70.11,56,0,3.49,PT,1659230338,Casa Maquia
2,2,3,Praia da Vitória,38.7333,-27.0667,71.85,88,0,4.47,PT,1659230413,Hotel Branco I
3,3,4,Fomboni,-12.2800,43.7425,75.63,72,0,3.83,KM,1659230422,Auberge les abou
4,4,5,Lagoa,39.0500,-27.9833,72.59,71,0,5.97,PT,1659230558,Casa Das Faias
5,5,6,Michigan City,41.7075,-86.8950,76.87,77,0,0.00,US,1659230541,Microtel Inn & Suites by Wyndham Michigan City
6,6,7,Hazorasp,41.3194,61.0742,71.44,37,0,8.68,UZ,1659230560,Kemping wifi hotel
7,7,8,Carbonia,39.1672,8.5222,75.85,74,0,6.98,IT,1659230633,Lu 'Hotel
8,8,9,Ajdabiya,30.7554,20.2263,78.75,70,0,9.57,LY,1659230633,Almadina Hotel
9,9,10,Marshfield,44.6689,-90.1718,74.28,64,0,4.61,US,1659230636,"Holiday Inn Conference Ctr Marshfield, an IHG ..."


In [93]:
# # Drop my cities that had no lodging within 5000 meters. 
# # Reran this code and then the code above and realized that since I was using the same named DF it deleted the empty rows
# # from it and now this is not necessary, so commented out.
# hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "No Hotel")]
# hotel_df.reset_index(inplace=True)
# # del IdealWeatherDF['index']
# hotel_df

In [87]:
# NOTE: Changed to match my colum headers.

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [95]:
# Add marker layer ontop of heat map
Markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
HeatMap.add_layer(Markers)
HeatMap

# Display figure


Figure(layout=FigureLayout(height='420px'))